# SPRINT16 課題論文読解入門

(1) 物体検出の分野にはどういった手法が存在したか。

- 【sliding window approach時代】：Deformable Parts Models(DPM)  
https://arxiv.org/abs/1409.5403　　
ある決まった大きさの領域を一定のピクセルごとにずらすことで領域候補を提案する手法　　
またHOG特徴と線形SVMを用いて認識対象物体の存在を判断する  
- 【region proposal method+deep learning時代】：R-CNN，Fast R-CNN  
https://arxiv.org/pdf/1506.01497.pdf  
畳み込みニューラルネットワークを利用した物体検出として中間層の出力から得られる特徴をCNN特徴と呼ぶ。  
回帰を使ってバウンディングボックスを予測しSVMで画像の分類をする。
- 【end-to-end時代 】：Faster R-CNN, YOLO, SSD  
https://arxiv.org/abs/1506.02640  
特徴マップの局所領域から計算される特徴ベクトルを回帰ネットワークと分類ネットワークで共有する。  
回帰ネットワークでは予め形状が決められたk個のアンカーボックスと呼ばれる入力の局所領域を中心とした標準的なバウンディングボックスを用意し  物体の有無の分類に関する損失とバウンディングボックスの位置とアスペクト比に関する損失を合わせたマルチタスク損失を最小化することでパラメータ学習をする。  
YOLOではセルごとに複数のバウンディングボックスを予測するが予測と正解のバウンディングボックスの一致度合いであるIoUで高い値を予測した予測器が検出対象物体に責任を持つ。


(2) Fasterとあるが、どういった仕組みで高速化したのか。

- Faster R-CNN：Fast R-CNNに対し、物体候補の領域の抽出にCNNベースのRegion Proposal Network (RPN) を利用することで、物体候補領域の高精度化を実現しつつ、真のend-to-end学習を実現    
  
要約に下記の説明あり  
The RPN is trained end-to-end to generate high-quality region proposals, which are used by Fast R-CNN for detection. We further merge RPN and Fast R-CNN into a single network by sharing their convolutional features—using the recently popular terminology of neural networks with “attention” mechanisms, the RPN component tells the unified network where to look.  

 RPNとFast R-CNNを畳み込み機能を共有することでさらに統合しています。最近注目されているニューラルネットワークの用語と「attention」メカニズムを使用して、RPNコンポーネントは統合ネットワークにどこを見ればよいかを伝えます。
 
 紹介文にも  
 INTRODUCTION  
 Recent advances in object detection are driven by　the success of region proposal methods (e.g., [4]) and region-based convolutional neural networks (RCNNs) [5]. Although region-based CNNs were computationally expensive as originally developed in [5], their cost has been drastically reduced thanks to sharing convolutions across proposals [1], [2]. The latest incarnation, Fast R-CNN [2], achieves near real-time rates using very deep networks [3], when ignoring the
time spent on region proposals. Now, proposals are the test-time computational bottleneck in state-of-the-art detection systems.

(3) One-Stageの手法とTwo-Stageの手法はどう違うのか。

1. DEEP LEARNING JP [DL Papers] “YOLO9000: Better, Faster, Stronger” (CVPR’17 Best Paper)  
And the History of Object Detection Makoto Kawano, Keio University  
http://deeplearning.jp/1

One-Stage  
一段のCNNで直接bounding boxの座標とクラスを推定する
- 代表的なモデルはYOLO, SSD
- two-stage detectorに比べると， 検出速度が速く，精度は劣る  


Two-Stage  
以下の2段階のプロセスで物体検出をする手法  
- ①候補領域の抽出
- ②クラス，bounding boxの座標の推定


物体検出の二つの時代  
• Not End-to-End Learning時代  
• 1.領域候補（Region Proposal）と2.物体認識(分類)を別々に行う  
• 物体認識で強かったCNNを取り入れてすごいとされた時代  
• End-to-End Learning時代  
• 1.と2.の処理を一つのニューラルネットワークで済ませる  
• 精度向上・速度向上を目指す時代  

(4) RPNとは何か。

上記に載せているように領域候補を出してそれを分類器にかけることで画像認識をするCNNのこと。分類にはSVMや回帰を用いる。  
https://arxiv.org/abs/1905.02758  
The Region Proposal Network (RPN) originally introduced for object detection within the popular  
Faster R-CNN is chosen as a reference DCNN. The reason is that a stand-alone RPN is able to serve as  
a competitive detector for two-class problems such as  
person detection. Furthermore, current state-of-the-art approaches initially apply  
an RPN followed by individual classifiers.とある。

(5) RoIプーリングとは何か。

RoIプーリングの詳細  
入力リストからすべての関心領域について、それに対応する入力特徴マップのセクションを取り、  
それを予め定義されたサイズ（例えば、7×7）にスケールする。スケーリングは次の方法で行う。  
領域候補を同じサイズのセクションに分割（その数は出力の次元と同じ）  
各セクションで最大値を見つける  
これらの最大値を出力バッファにコピーする  
その結果、サイズの異なる長方形のリストから、固定サイズの対応する特徴マップのリストをすばやく取得できる。  
RoIプーリング出力の次元は、実際には入力特徴マップのサイズや領域提案のサイズに依存しないことに注意。  
領域候補を分割するセクションの数だけによって決定される。  
Figure 1: Key idea of R-FCN for object detection. In this illustration, there are k × k = 3 × 3  
position-sensitive score maps generated by a fully convolutional network. For each of the k × k bins  
in an RoI, pooling is only performed on one of the k 2 maps (marked by different colors).

<img src="screen2.png">

(6) Anchorのサイズはどうするのが適切か。

バウンディングボックスを予め用意しておきそのボックスに対してアスペクト比を可変にしたものを縦横複数用意しておきアンカーとして  
物体にある程度許容をして物体検知をする。

(7) 何というデータセットを使い、先行研究に比べどういった指標値が得られているか。

accuracy on PASCAL VOC 2007, 2012, and MS COCO datasets  
with only 300 proposals per image. In ILSVRC and COCO 2015 competitions  
論文に上記の記載があり 

http://cocodataset.org/#home  
にデータセットの説明がある。  
詳しくは  
オブジェクト分割  
文脈における認識  
スーパーピクセルのもののセグメンテーション  
330K画像（> 200Kラベル付き）  
150万個のオブジェクトインスタンス  
80のオブジェクトカテゴリ  
91ものカテゴリ  
画像あたり5つのキャプション  
キーポイントを持つ25万人  
http://host.robots.ox.ac.uk/pascal/VOC/voc2012/  
Example images  
Example images and the corresponding annotation for the classification/detection/segmentation/action     tasks, and person layout taster can be viewed online:  

Classification/detection example images  
Segmentation example images  
Action Classification example images  
Person Layout taster example images  

<img src="screen.png">

mAPとは、mean Average Precisionの略で、平均適合率の平均。検出した物体が正しいかどうかを示し、数値が大きければ大きいほど精度が高い。  
Using the expensive very deep models of [3], our detection method still has a frame rate of 5fps  (including all steps) on a GPU,and thus is a practical object detection system in terms of both speed and accuracy.  
  
[3] の高価で非常に深いモデルを使用して、我々の検出方法は、GPU 上の 5fps (すべてのステップを含む) のフレームレートをまだ持っており、したがって、速度と精度の両方の面で実用的なオブジェクト検出システムである。